In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# December 16 games
sixers_ml = MoneyLine(event="sixers_ml", bet_amount=100, odds=-286) 
heat_ml = MoneyLine(event="heat_ml", bet_amount=100, odds=235)

over = MoneyLine(event="over", bet_amount=100, odds=-112)
under = MoneyLine(event="under", bet_amount=100, odds=-104)

binaries = [
    [sixers_ml, heat_ml],
    [over_217, under_217],
]

results = [
    [0, 1], #[sixers_ml, heat_ml],
    [0, 1], #[over_217, under_217],
]

def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 2
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

1

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=50,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -1.3233621428684086
     jac: array([-0.04291958,  0.12664455, -1.21812499, -0.79724576])
 message: 'Positive directional derivative for linesearch'
    nfev: 50
     nit: 9
    njev: 5
  status: 8
 success: False
       x: array([0.22165726, 0.01      , 0.68627489, 0.60100232])
slsqp_solver: 
                 event index[]  result  event_status   bet      mult  payout  \
3    heat_ml_under_217  [1, 3]  [1, 1]          True  0.60  6.188983  3.7196   
2     heat_ml_over_217  [1, 2]  [1, 0]         False  0.69  7.872500  5.4027   
0   sixers_ml_over_217  [0, 2]  [0, 0]         False  0.22  3.171678  0.7030   
1  sixers_ml_under_217  [0, 3]  [0, 1]         False  0.01  2.493422  0.0249   

   profit  
3    2.20  
2    3.88  
0   -0.82  
1   -1.49  
total_bet   :  1.52
mean        :  0.9424999999999999
max         :  3.88
min         :  -1.49
std         :  2.532039691632025
            bet      mult    payout   profit
count  4.000000  4.000000  4.000000  4.00000
mean   0.380000 

In [8]:
df.describe()

,bet,mult,payout,profit
count,4.000000,4.000000,4.000000,4.00000
mean,0.380000,4.931646,2.462550,0.94250
std,0.319896,2.534557,2.533957,2.53204
min,0.010000,2.493422,0.024900,-1.49000
25%,0.167500,3.002114,0.533475,-0.98750
50%,0.410000,4.680331,2.211300,0.69000
75%,0.622500,6.609862,4.140375,2.62000
max,0.690000,7.872500,5.402700,3.88000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,2.000000,2.000000,2.000000,2.000000
mean,0.115000,2.832550,0.363950,-1.155000
std,0.148492,0.479600,0.479489,0.473762
min,0.010000,2.493422,0.024900,-1.490000
25%,0.062500,2.662986,0.194425,-1.322500
50%,0.115000,2.832550,0.363950,-1.155000
75%,0.167500,3.002114,0.533475,-0.987500
max,0.220000,3.171678,0.703000,-0.820000


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,2.00000,2.000000,2.000000,2.000000
mean,0.64500,7.030742,4.561150,3.040000
std,0.06364,1.190426,1.190131,1.187939
min,0.60000,6.188983,3.719600,2.200000
25%,0.62250,6.609862,4.140375,2.620000
50%,0.64500,7.030742,4.561150,3.040000
75%,0.66750,7.451621,4.981925,3.460000
max,0.69000,7.872500,5.402700,3.880000


In [11]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
1,sixers_ml_under_217,"[0, 3]","[0, 1]",False,0.01,2.493422,0.0249,-1.49
0,sixers_ml_over_217,"[0, 2]","[0, 0]",False,0.22,3.171678,0.7030,-0.82


In [12]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
3,heat_ml_under_217,"[1, 3]","[1, 1]",True,0.60,6.188983,3.7196,2.20
2,heat_ml_over_217,"[1, 2]","[1, 0]",False,0.69,7.872500,5.4027,3.88


In [13]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,1,1,1,1,1,1,1
True,1,1,1,1,1,1,1


In [14]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,2,2,2,2,2,2,2


In [15]:
#csv

In [16]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'red' if val == True else 'black'
    return 'color: %s' % color

def color_positive_green(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'green' if val == True else 'black'
    return 'color: %s' % color

In [17]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,bet,mult,payout,profit
0,sixers_ml_under_217,"['0', '3']","[0, 1]",False,0.01,2.49342,0.0249,-1.49
1,sixers_ml_over_217,"['0', '2']","[0, 0]",False,0.22,3.17168,0.703,-0.82


In [18]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,bet,mult,payout,profit
0,heat_ml_under_217,"['1', '3']","[1, 1]",True,0.6,6.18898,3.7196,2.2
1,heat_ml_over_217,"['1', '2']","[1, 0]",False,0.69,7.8725,5.4027,3.88
